In [577]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import math
from datetime import datetime, timedelta
import statistics 
from statistics import median

In [578]:
fre1 = pd.read_csv("frequency_20180101_20180301.csv")
fre2 = pd.read_csv("frequency_20180301_20180701.csv")
fre3 = pd.read_csv("frequency_20180701_20180711.csv")
fre = pd.concat([fre1,fre2,fre3])
fre.index = range(len(fre))
win = pd.read_csv("wind_generation.csv")
thr = pd.read_csv("throughput_ratio.csv")

In [579]:
#set the row number of data that you want to process considering your running time
#Good Luck!
#There are two dataframes, df_raise for Raise Throughput Ratio and df_lower for Lower Throughput Ratio
n=1000;

In [580]:
df=win
df["Frequency (Hz)"] = np.nan
df["Raise Throughput Ratio"] = np.nan
df["Lower Throughput Ratio"] = np.nan
df["Median Frequency (Hz)"] = np.nan

In [581]:
thr_time=thr['Interval Ending (AEST)'].tolist()
fre_time=[round(t/1000) for t in fre['Timestamp (GMT)'].tolist()]

In [582]:
avg_time=[]
avg_fre=[]
med_fre=[]
tem_fre=[]
avg=0
num=0
##for i in range(len(fre_time)):
for i in range(60000,70000+300*n):
    tem_fre.append(float(fre.loc[i,["Frequency (Hz)"]]))
    if fre_time[i]%300==0:
        avg_fre.append(sum(tem_fre)/len(tem_fre))
        med_fre.append(median(tem_fre))
        avg_time.append(fre_time[i])
        tem_fre=[]

In [583]:
print('start time is',df["Interval Ending (AEST)"][0],  \
      '\nstart timestamp is',(datetime.strptime('1/1/18 0:05', '%m/%d/%y %H:%M')-timedelta(hours=18)).timestamp(),\
      '\nstat index in frequency is',fre_time.index(1514793900.0))

start time is 1/1/18 0:05 
start timestamp is 1514729100.0 
stat index in frequency is 65100


In [584]:
for i in range(n):
    df_tstamp=((datetime.strptime(df["Interval Ending (AEST)"][i], '%m/%d/%y %H:%M')- timedelta(hours=18)).timestamp())
    if df_tstamp in avg_time:
        index=avg_time.index(df_tstamp)
        df.loc[i,["Frequency (Hz)"]]=avg_fre[index]
        df.loc[i,["Median Frequency (Hz)"]]=med_fre[index]
    if df["Interval Ending (AEST)"][i] in thr_time:
        index=thr_time.index(df["Interval Ending (AEST)"][i])
        df.loc[i,["Raise Throughput Ratio"]]=thr.loc[index,["Raise Throughput Ratio"]]
        df.loc[i,["Lower Throughput Ratio"]]=thr.loc[index,["Lower Throughput Ratio"]]

In [585]:
df_lower=df.iloc[0:n,:].drop(['Raise Throughput Ratio'], axis=1)
df_lower.head()
s=pd.isna(df_lower['Lower Throughput Ratio'])
for i in range(len(s)):
    if s[i]:
        df_lower=df_lower.drop(i)

In [586]:
df_raise=df.iloc[0:n,:].drop(['Lower Throughput Ratio'], axis=1)
df_raise.head()
s=pd.isna(df_raise['Raise Throughput Ratio'])
for i in range(len(s)):
    if s[i]:
        df_raise=df_raise.drop(i)

In [587]:
df_raise['Frequency (Hz)'].mean()

49.997579301982874

In [588]:
df_lower['Frequency (Hz)'].mean()

50.017552012043076

In [589]:
df_lower.to_csv('df_lower.csv')

In [590]:
df_raise.to_csv('df_raise.csv')